In [1]:
#install langchain
!pip install langchain

In [2]:
#install openAi
!pip install openai

In [3]:
#install logger
pip install loguru


                                              0.0/60.0 kB ? eta -:--:--
     ---------------------------------------- 60.0/60.0 kB 3.1 MB/s eta 0:00:00


In [4]:
#install redis
pip install redis

                                              0.0/241.1 kB ? eta -:--:--
     -------------------------------------- 241.1/241.1 kB 7.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
# import redis
from langchain.memory import RedisChatMessageHistory

history = RedisChatMessageHistory("New")

In [6]:
from loguru import logger

from langchain.callbacks import FileCallbackHandler
#logging into the file
logfile = "./mermaid/outputFlowDiaNew.log"

logger.add(logfile, colorize=True, enqueue=True)
handler = FileCallbackHandler(logfile)

In [7]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [8]:
from langchain.callbacks import get_openai_callback
#count tokens
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        count = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return count

In [29]:
from langchain import OpenAI
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

modell = "gpt-3.5-turbo"

# first initialize the large language model
llm = ChatOpenAI(model= modell, temperature=0.7, max_tokens=1024, top_p=1, openai_api_key=os.getenv("OPENAI_API_KEY"))

# now initialize the conversation chain
conversation = ConversationChain(llm=llm)

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


In [30]:
systemInput = "Convert the following text to MermaidJS markup, do not explain and give the exact syntax : ";
userInput = "For christmas shopping, we need first need to have some money. Then we go shopping, and then think what we want to buy. The things we can buy are iPhone, car or laptop."
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [31]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [45]:
print(systemInput + userInput)
result = conversation_buf(systemInput + userInput)
tokens = count_tokens(conversation_buf, (systemInput + userInput))
#adding Human messages and AI messages
history.add_user_message(result['input'])
history.add_ai_message(result['response'])
logger.info(result)  


Convert the following text to MermaidJS markup, do not explain and give the exact syntax : For christmas shopping, we need first need to have some money. Then we go shopping, and then think what we want to buy. The things we can buy are iPhone, car or laptop.


2023-08-03 12:37:27.859 | INFO     | __main__:<module>:6 - {'input': 'Convert the following text to MermaidJS markup, do not explain and give the exact syntax : For christmas shopping, we need first need to have some money. Then we go shopping, and then think what we want to buy. The things we can buy are iPhone, car or laptop.', 'history': 'Human: Convert the following text to MermaidJS markup, do not explain and give the exact syntax : For christmas shopping, we need first need to have some money. Then we go shopping, and then think what we want to buy. The things we can buy are iPhone, car or laptop.\nAI: ```mermaid\ngraph LR\nA((Start)) --> B(Have money)\nB --> C(Go shopping)\nC --> D(Think about what to buy)\nD --> E{iPhone}\nD --> F[Car]\nD --> G[Laptop]\n```\n\nHuman: Convert the following text to MermaidJS markup, do not explain and give the exact syntax : For christmas shopping, we need first need to have some money. Then we go shopping, and then think what we want to buy. The

Spent a total of 748 tokens


In [46]:
print(result['response'])

```mermaid
graph LR
A((Start)) --> B(Have money)
B --> C(Go shopping)
C --> D(Think about what to buy)
D --> E{iPhone}
D --> F[Car]
D --> G[Laptop]
```


In [47]:
import base64
import requests
import io
from PIL import Image

graph=result['response']

print(graph)



# Find the index of the first occurrence of "graph LR"
start_index = graph.find("graph")

# Find the index of the last occurrence of "]" after "D --> G[Laptop]"
end_index = graph.rfind("]", graph.find("D --> G[Laptop]"))

# Extract the desired part of the string
desired_part = graph[start_index:end_index + 1]

graphbytes = desired_part.encode("ascii")



base64_bytes = base64.b64encode(graphbytes)
base64_string = base64_bytes.decode("ascii")

img = Image.open(io.BytesIO(requests.get('https://mermaid.ink/img/' + base64_string).content))
img.show()
logger.info("Flow Diagram generated Successfully, Model : " + modell + " and Total Tokens : " + tokens)




```mermaid
graph LR
A((Start)) --> B(Have money)
B --> C(Go shopping)
C --> D(Think about what to buy)
D --> E{iPhone}
D --> F[Car]
D --> G[Laptop]
```


2023-08-03 12:37:40.024 | INFO     | __main__:<module>:30 - Flow Diagram generated Successfully, Model : gpt-3.5-turbo and Total Tokens : ```mermaid
graph LR
A((Start)) --> B(Have money)
B --> C(Go shopping)
C --> D(Think about what to buy)
D --> E{iPhone}
D --> F[Car]
D --> G[Laptop]
```


In [48]:
history.messages

[HumanMessage(content='Convert the following text to MermaidJS markup, do not explain and give the exact syntax : For christmas shopping, we need first need to have some money. Then we go shopping, and then think what we want to buy. The things we can buy are iPhone, car or laptop.', additional_kwargs={}, example=False),
 AIMessage(content='The MermaidJS markup for the given text would be as follows:\n\n```\ngraph LR\nA[Christmas shopping] --> B[Have money]\nB --> C[Go shopping]\nC --> D[Think what to buy]\nD --> E[iPhone]\nD --> F[Car]\nD --> G[Laptop]\n```\n\nThis markup represents a flowchart where "Christmas shopping" is the starting point, and it is connected to "Have money" through an arrow. "Have money" is connected to "Go shopping," which is then connected to "Think what to buy." From "Think what to buy," there are three options to choose from: "iPhone," "Car," and "Laptop," each connected through separate arrows.', additional_kwargs={}, example=False),
 HumanMessage(content='C